# Using Chroma

The goal of this colab is to have a vector database like chroma to create embeddings of new documents, and store them, create a doc searcher, and use that to talk to documents.

### install libraries
This version uses llama_hubs vectore store and default query engine modules.
the underlying model that is called is openai's gpt3.5 model but can be changed to anything else

In [1]:
%pip install llama_index=="0.8.41"
%pip install llama_hub=="0.0.36"
%pip install youtube_transcript_api=="0.6.1"
%pip install chromadb=="0.4.15"


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.4/473.4 kB 5.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 24.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━

### configuration of modules

- logging and required openai api key

In [ ]:
import os
import logging
import sys
import getpass

logging.basicConfig(stream=sys.stdout, level=logging.WARN)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

### download youtube video's transcripts

- using llama_hub's [youtube transcript loader] (https://llamahub.ai/l/youtube_transcript)

In [ ]:
from llama_hub.youtube_transcript import YoutubeTranscriptReader

loader = YoutubeTranscriptReader()
documents = loader.load_data(ytlinks=['https://www.youtube.com/watch?v=VH8lEOmesas','https://www.youtube.com/watch?v=ZSEnQ00iUgI'])


### index in chroma with embeddings

- using llama_hubs vectorstore wrapper

In [ ]:
import chromadb
from chromadb.config import Settings
from llama_index import VectorStoreIndex, StorageContext
from llama_index.vector_stores import ChromaVectorStore

chromadb_settings = Settings(anonymized_telemetry=False)
client = chromadb.Client(settings=chromadb_settings)
collection = client.get_or_create_collection(name="test")
chromastore = ChromaVectorStore(chroma_collection=collection)

index = VectorStoreIndex.from_documents(
    documents=documents,
    storage_context=StorageContext.from_defaults(vector_store=chromastore)
)


# use query engine to talk to docs

this uses the vector store wrapper from [llama_index](https://gpt-index.readthedocs.io/en/v0.6.5/how_to/integrations/vector_stores.html)

In [29]:
import textwrap

query_engine = index.as_query_engine()
question = "I still run old Softare and Services in my company. Who you gonna call?"         
response = query_engine.query(question)

wrapped_lines = textwrap.wrap(response.response, 79, break_long_words=False)

for line in wrapped_lines:
  print(line)

You should contact the Deep Shore Cloud Archive with microservices. They have
developed a solution that can turn your traditional archive into a real-time
enabled Enterprise Information Catalog. Their services offer unlimited access
from anywhere in the world, full regulatory compliance and security, and
scalability in bandwidth and data volume. They can also feed ERP systems and
data warehouses with the complete digital knowledge of your business in real
time.
